In [1]:
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt


In [ ]:
ds = "iv_recording"
clip_id = "C1038--0001"
bucket_id = 32
img_folder = Path(f"/data/lipsync_512_data/")
bucket_folder = img_folder / ds / clip_id / f"{bucket_id:06d}"
img_list = list(bucket_folder.glob("*.png"))
img_list.sort()
print(img_list)
img_list = [cv2.imread(str(img)) for img in img_list]



In [ ]:
image = np.concatenate(img_list, 1)
plt.imshow(image[:, :, ::-1])
plt.show()


In [ ]:
lms_folder = Path("/data/mp_lms")
lms = np.load(lms_folder / f"{ds}/{clip_id}.npy")
print(lms.shape)


In [ ]:
bbox_folder = Path("/data/lipsync_512_data/bins")
bbox = np.load(bbox_folder / f"{ds}/{clip_id}.npy")
print(bbox.shape)


In [ ]:
bbox[bucket_id]

In [ ]:
lms[162].astype(np.int32)


In [15]:
FACEMESH_LIPS = frozenset([(61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
                           (17, 314), (314, 405), (405, 321), (321, 375),
                           (375, 291), (61, 185), (185, 40), (40, 39), (39, 37),
                           (37, 0), (0, 267),
                           (267, 269), (269, 270), (270, 409), (409, 291),
                           (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
                           (14, 317), (317, 402), (402, 318), (318, 324),
                           (324, 308), (78, 191), (191, 80), (80, 81), (81, 82),
                           (82, 13), (13, 312), (312, 311), (311, 310),
                           (310, 415), (415, 308)])


In [ ]:
def draw_lips(img_folder, lms_folder, bbox_folder, ds, clip_id, bucket_id):
    print(img_folder, lms_folder, bbox_folder, ds, clip_id, bucket_id)
    img_folder = Path(img_folder)
    lms_folder = Path(lms_folder)
    bbox_folder = Path(bbox_folder)
    img_list = list((img_folder / ds / clip_id / f"{bucket_id:06d}").glob("*.png"))
    img_list.sort()
    img_list = [cv2.imread(str(img)) for img in img_list]
    h, w = img_list[0].shape[:2]
    lms = np.load(lms_folder / f"{ds}/{clip_id}.npy")
    bbox = np.load(bbox_folder / f"{ds}/{clip_id}.npy")
    bbox = bbox[bucket_id]
    for i in range(bbox.shape[0]):
        lm = lms[bbox[i][1]]
        lm = lm.astype(np.int32)
        bbox_h, bbox_w = bbox[i][4] - bbox[i][2], bbox[i][5] - bbox[i][3]
        lm[:, 0] = (lm[:, 0] - bbox[i][2]) / bbox_w * w
        lm[:, 1] = (lm[:, 1] - bbox[i][3]) / bbox_h * h
        lm = lm.astype(np.int32)
        for (p1, p2) in FACEMESH_LIPS:
            cv2.line(img_list[i], (lm[p1][0], lm[p1][1]), (lm[p2][0], lm[p2][1]), (0, 255, 0), 2)
    
    image = np.concatenate(img_list, 1)
    cv2.imwrite("test.png", image)
    plt.imshow(image[:, :, ::-1])
    plt.show()

draw_lips('/data/lipsync_512_data/', '/data/mp_lms', '/data/lipsync_512_data/bins', 'iv_recording', 'C1038--0001', 32)



In [ ]:
import face_alignment
from skimage import io
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.THREE_D, device='cuda')

ds = "iv_recording"
clip_id = "C1038--0001"
bucket_id = 32
img_folder = Path(f"/data/lipsync_512_data/")
bucket_folder = img_folder / ds / clip_id / f"{bucket_id:06d}"
img_list = list(bucket_folder.glob("*.png"))
img_list.sort()
img_list = [io.imread(str(img)) for img in img_list]

for img in img_list:
    landmarks = fa.get_landmarks(img)
    print(landmarks[0].shape)
